In [1]:
// #load "NbHelper.fsx"

#I "../../../Vide.Common/src/Vide.Common/bin/Debug/netstandard2.0"
#r "Vide.Common.dll"

#r "nuget: Plotly.NET, 4.2.0"
#r "nuget: Plotly.NET.Interactive, 4.2.0"

Installed Packages Plotly.NET, 4.2.0 Plotly.NET.Interactive, 4.2.0

Loading extensions from `C:\Users\ronal\.nuget\packages\plotly.net.interactive\4.2.0\interactive-extensions\dotnet\Plotly.NET.Interactive.dll`

In [2]:
open Vide
open Vide.Dsp
open Vide.Dsp.Operators
open Vide.Dsp.DiscreteTime

open Plotly.NET
open Plotly.NET.LayoutObjects

let mkLineChartXY xvalues (yvalues: _ list) =
    Chart.Line(xvalues, yvalues)
    |> Chart.withSize(800.0, 300.0)
    |> Chart.withMargin(Margin.init(5,5,15,5))

let mkLineChart (yvalues: _ list) =
    let xvalues = List.init (yvalues |> List.length) float
    mkLineChartXY xvalues yvalues 

let mkGenChart sampleRate durationInSeconds (gen: Vide<_,_,_>) =
    let yvalues =
        gen 
        |> Gen.Eval.toSeq sampleRate
        |> Seq.take (sampleRate * durationInSeconds |> int)
        |> Seq.toList
    let xvalues = List.init yvalues.Length (fun y -> float y / sampleRate)
    mkLineChartXY xvalues yvalues
    
let mkGensChart sampleRate durationInSeconds gens =
    gens
    |> List.map (mkGenChart sampleRate durationInSeconds)
    |> Chart.combine

let mkGenChart44kHz durationInSeconds gen = mkGenChart 44100.0 durationInSeconds gen
    
let mkGensChart44kHz durationInSeconds gens = mkGensChart 44100.0 durationInSeconds gens

In [ ]:
// showLine [ 0. .. 10. ]
Gen.Osc.sin 40.0 |> mkGenChart44kHz 1.0

In [4]:
[
    Gen.Osc.noise |> mkGenChart44kHz 1.0
    Gen.Osc.noise |=> Fx.Filter.lowPass { frq = 100.0; q = 0.5; gain = 1.0 } |> mkGenChart44kHz 1.0
]
|> Chart.combine


<!-- Plotly chart will be drawn inside this DIV -->